#### `RAG = Retrieval-Augmented Generation`

It’s an architecture that combines information retrieval (like search) with generative AI (like LLMs such as GPT, LLaMA, etc.) to answer user queries with context-aware, accurate, and source-grounded responses.

## RAG Architecture

## Chroma 

`Chroma` is an AI-native open-source `vector database` It comes with everything you need to get started built in, and runs on your machine. A hosted version is now available for early access!

In [1]:
import os
from groq import Groq
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import glob

C:\Users\Rohan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
GROQ_API_KEY = "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"  
client = Groq(api_key=GROQ_API_KEY)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
chroma_client = chromadb.PersistentClient(path="./vector_db")
collection = chroma_client.get_or_create_collection(name="company_docs")

base_path = "data"
doc_id = 0

for folder in os.listdir(base_path):
    role_path = os.path.join(base_path, folder)
    if os.path.isdir(role_path):
        for file in glob.glob(f"{role_path}/*.md"):
            with open(file, "r", encoding="utf-8") as f:
                text = f.read()
            embedding = embedding_model.encode(text).tolist()
            collection.add(
                documents=[text],
                embeddings=[embedding],
                metadatas=[{"role": folder, "source": os.path.basename(file)}],
                ids=[f"doc_{doc_id}"]
            )
            doc_id += 1

print("✅ All documents embedded and stored.")

✅ All documents embedded and stored.


In [4]:
def retrieve_docs(query, user_role, top_k=3):
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        where={"role": user_role}
    )
    docs = results['documents'][0]
    metadatas = results['metadatas'][0]
    sources = [md['source'] for md in metadatas]
    return docs, sources


In [5]:
def ask_groq_rag(query, user_role):
    docs, sources = retrieve_docs(query, user_role)
    context = "\n\n".join(docs)

    prompt = f"""You are a helpful assistant answering questions for the {user_role} department.

Use only the context below to answer the user's question.

Context:
{context}

Question:
{query}
"""

    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": f"You are a helpful assistant for the {user_role} team."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content, sources


In [6]:
def chat_loop(user_role):
    print(f"💬 Chatbot active for role: {user_role.capitalize()}")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("👋 Chatbot: Goodbye!")
            break
        response, sources = ask_groq_rag(user_input, user_role)
        print(f"\n🧠 Chatbot:\n{response}")
        print(f"\n📎 Sources: {', '.join(sources)}\n{'-'*50}")


In [ ]:
chat_loop("finance")

💬 Chatbot active for role: Finance
Type 'exit' to quit.

